In [9]:
import mysql.connector
import pymysql

from mysql.connector import Error

**Ceedenciales**

In [ ]:


try:
    con = mysql.connector.connect(
        host="database-project.clsk6e68ec80.eu-north-1.rds.amazonaws.com",
        user="admin",
        password="holamundo1",
        database="tumor_database"   # 👈 nombre correcto
    )

    if con.is_connected():
        cursor = con.cursor()
        print("Conexión exitosa a tumor_database en AWS RDS")

except Error as e:
    print(" Error al conectar:", e)



Conexión exitosa a tumor_database en AWS RDS


**Conexion A DB**

In [27]:
import pymysql

# 🔒 CONFIGURACIÓN CORRECTA PARA AWS RDS
host = "database-project.clsk6e68ec80.eu-north-1.rds.amazonaws.com"
user = "admin"
password = "holamundo1"
database_name = "tumor_database"

# 🔌 CREAR CONEXIÓN
db = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database_name,
    cursorclass=pymysql.cursors.DictCursor
)

cursor = db.cursor()
print("Conexión OK ✔")


Conexión OK ✔


**Version DB**

In [28]:
cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL instance version: {version}') 

MySQL instance version: {'VERSION()': '8.0.43'}


**Creacion DB**

In [29]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE IF NOT EXISTS tumor_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
delete_db = '''DROP DATABASE tumor_database'''
cursor.execute(delete_db)

# La volvemos a crear
cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [30]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'},
 {'Database': 'tumor_database'}]

**Seleccionar DB**

In [31]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE tumor_database'''
cursor.execute(use_db)

0

**Creacion de tablas**

In [33]:
# 🧩 Función para conectar a BD
def conectar():
    try:
        con = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database_name,
            cursorclass=pymysql.cursors.DictCursor
        )
        print("Conexión exitosa a AWS RDS ✔")
        return con
    except Error as e:
        print(" Error al conectar:", e)
        return None

In [34]:

# 🏗️ Creación de tablas
def crear_tablas():
    con = conectar()
    cursor = con.cursor()

    queries = [
        """
        CREATE TABLE IF NOT EXISTS images (
            image_id INT AUTO_INCREMENT PRIMARY KEY,
            file_name VARCHAR(255),
            s3_url VARCHAR(500) NOT NULL,
            uploaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS predictions (
            pred_id INT AUTO_INCREMENT PRIMARY KEY,
            image_id INT NOT NULL,
            tumor_detected BOOLEAN,
            tumor_type VARCHAR(100),
            confidence FLOAT,
            predicted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (image_id) REFERENCES images(image_id)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS segmentations (
            seg_id INT AUTO_INCREMENT PRIMARY KEY,
            image_id INT NOT NULL,
            mask_s3_url VARCHAR(500),
            dice_score FLOAT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (image_id) REFERENCES images(image_id)
        )
        """
    ]

    for query in queries:
        cursor.execute(query)
        print("Tabla creada correctamente")

    con.commit()


crear_tablas()

Conexión exitosa a AWS RDS ✔
Tabla creada correctamente
Tabla creada correctamente
Tabla creada correctamente


In [35]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_tumor_database': 'images'},
 {'Tables_in_tumor_database': 'predictions'},
 {'Tables_in_tumor_database': 'segmentations'}]

**Insertar Datos en las tabalas**

In [36]:
cursor.execute("""
INSERT INTO images (file_name, s3_url)
VALUES (%s, %s)
""", ("MRI_001.png", "https://bucket/MRI_001.png"))

con.commit()

image_id = cursor.lastrowid
print("ID de la imagen:", image_id)


ID de la imagen: 1


In [37]:
cursor.execute("""
INSERT INTO predictions (image_id, tumor_detected, tumor_type, confidence)
VALUES (%s, %s, %s, %s)
""", (image_id, True, "Glioma", 0.94))
con.commit()


In [38]:
cursor.execute("""
INSERT INTO segmentations (image_id, mask_s3_url, dice_score)
VALUES (%s, %s, %s)
""", (image_id, "https://bucket/MRI_001_mask.png", 0.87))
con.commit()


**Leer Datos**

In [39]:
sql = '''SELECT * FROM tumor_database.images'''
cursor.execute(sql)

1

In [40]:
cursor.fetchall()

[{'image_id': 1,
  'file_name': 'MRI_001.png',
  's3_url': 'https://bucket/MRI_001.png',
  'uploaded_at': datetime.datetime(2025, 12, 4, 20, 55, 4)}]

1